In [1]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir2='/Volumes/Extreme Pro/'
dir1='/Volumes/Extreme Pro 1/'
dir_data='/Users/lexihenny/'

/var/folders/ql/bxfj9fdd15xfw55_bmktgrf80000gn/T/ipykernel_51941/1876262561.py:23: DeprecationWarning: Please use `label` from the `scipy.ndimage` namespace, the `scipy.ndimage.measurements` namespace is deprecated.
  from scipy.ndimage.measurements import label


In [ ]:
if 1==0:
    ds=xr.open_dataset(dir_data+'synoptic_geochars_jra55gw_MAX.nc')
    ivt_concat=ds['ivt_concat'].values
    iwv_concat=ds['iwv_concat'].values
    s_concat=ds['s_concat'].values
    mfc_concat=ds['mfc_concat'].values
    print(len(ivt_concat[:,0,0]))

yrs=np.arange(1980,2020,1)
for i in range(0,len(yrs)):
    year=yrs[i]
    print(year)
    
    if 1==0:
        ds=xr.open_dataset(dir2+'era_5_ivtinterp_'+str(year)+'.nc')
        ivt=ds['ivt'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_iwvinterp_'+str(year)+'.nc')
        iwv=ds['iwv'][:,:,:]
        ds=xr.open_dataset(dir2+'era_5_u850interp_'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir2+'era_5_v850interp_'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)

        ds=xr.open_dataset(dir2+'era_5_mfcinterp_'+str(year)+'.nc')#era5 is divergence
        mfc=ds['mfc'][:,:,:]*-1.
    if 1==0:
        ds=xr.open_dataset(dir2+'merra2.ivt.3hr.model.'+str(year)+'.nc')
        ivt=ds['ivt'][0::2,:,:]
        ds=xr.open_dataset(dir2+'merra2.iwv.3hr.model.'+str(year)+'.nc')
        iwv=ds['iwv'][0::2,:,:]
        ds=xr.open_dataset(dir1+'merra2.u850.6hr.'+str(year)+'.nc')
        u=ds['u']
        ds=xr.open_dataset(dir1+'merra2.v850.6hr.'+str(year)+'.nc')
        v=ds['v']
        s=np.sqrt(u**2+v**2)
        
        ds=xr.open_dataset(dir2+'merra2.mfc.3hr.model.'+str(year)+'.nc')#merra-2 is tendency
        mfc=ds['mfc'][0::2,:,:]
        
    ds=xr.open_dataset(dir2+'jra55.ar.gwv2.'+str(year)+'.nc')
    ar=ds['ar']
    
    if 1==1:
        ds=xr.open_dataset(dir1+'jra55_ivt_'+str(year)+'.nc')
        ivt=ds['ivt'].interp(lat=ar.lat.values,lon=ar.lon.values)
        ds=xr.open_dataset(dir1+'jra55_iwv_'+str(year)+'.nc')
        iwv=ds['iwv'].interp(lat=ar.lat.values,lon=ar.lon.values)
        ds=xr.open_dataset(dir1+'jra55_u850_'+str(year)+'.nc')
        u=ds['u'].interp(lat=ar.lat.values,lon=ar.lon.values)
        ds=xr.open_dataset(dir1+'jra55_v850_'+str(year)+'.nc')
        v=ds['v'].interp(lat=ar.lat.values,lon=ar.lon.values)
        s=np.sqrt(u**2+v**2)
        
    #ds=xr.open_dataset(dir2+'era5.ar.tempestlr'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.reid500.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'era5.ar.mundhenkv3.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.lorav2.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.cndl.'+str(year)+'.nc')
    #ds=xr.open_dataset(dir2+'merra2.ar.acv2.'+str(year)+'.nc')
    #ar=ds['ar']
    
    ivt['lon']=ar.lon.values
    iwv['lon']=ar.lon.values
    s['lon']=ar.lon.values
    #mfc['lon']=ar.lon.values
        
    ivt['lat']=ar.lat.values
    iwv['lat']=ar.lat.values
    s['lat']=ar.lat.values
    #mfc['lat']=ar.lat.values
    
    ivt['time']=ar.time.values
    iwv['time']=ar.time.values
    s['time']=ar.time.values
    #mfc['time']=ar.time.values
    
    #weights=np.cos(np.deg2rad(ar.lat)
    
    ivt=ivt.where(ar==1)
    ivt_mean=ivt.max(dim='time',skipna=True).values[np.newaxis,:,:]
    iwv=iwv.where(ar==1)
    iwv_mean=iwv.max(dim='time',skipna=True).values[np.newaxis,:,:]
    s=s.where(ar==1)
    s_mean=s.max(dim='time',skipna=True).values[np.newaxis,:,:]
    #mfc=mfc.where(ar==1)
    #mfc_mean=mfc.max(dim='time',skipna=True).values[np.newaxis,:,:]
    
    if 1==0:
        ivt_sel1=ivt[0,:,:]
        fig=plt.figure(figsize=(18,12))
        ax=plt.sub72ot(1,1,1,projection=ccrs.PlateCarree())
        ax.coastlines(resolution='10m')
        ax.pcolormesh(ivt_sel1.lon.values,ivt_sel1.lat.values,ivt_sel1,vmin=0,vmax=800,cmap=plt.cm.seismic)
        plt.show()

        iwv_sel1=iwv[0,:,:]
        fig=plt.figure(figsize=(18,12))
        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
        ax.coastlines(resolution='10m')
        ax.pcolormesh(ivt_sel1.lon.values,ivt_sel1.lat.values,iwv_sel1,vmin=0,vmax=60,cmap=plt.cm.seismic)
        plt.show()
           
        s_sel1=s[0,:,:]
        fig=plt.figure(figsize=(18,12))
        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
        ax.coastlines(resolution='10m')
        ax.pcolormesh(ivt_sel1.lon.values,ivt_sel1.lat.values,s_sel1,vmin=0,vmax=30,cmap=plt.cm.seismic)
        plt.show()
        sys.exit()
        
    if i==0:
        ivt_concat=ivt_mean
        iwv_concat=iwv_mean
        s_concat=s_mean
        #mfc_concat=mfc_mean
    else:
        
        ivt_concat=np.concatenate((ivt_concat,ivt_mean),axis=0)
        iwv_concat=np.concatenate((iwv_concat,iwv_mean),axis=0)
        s_concat=np.concatenate((s_concat,s_mean),axis=0)
        #mfc_concat=np.concatenate((mfc_concat,mfc_mean),axis=0)

In [ ]:
dir_data='/Users/lexihenny/'
dk=xr.Dataset()
dk['ivt_concat']=(('time','lat','lon'),ivt_concat)
dk['iwv_concat']=(('time','lat','lon'),iwv_concat)
dk['s_concat']=(('time','lat','lon'),s_concat)
#dk['mfc_concat']=(('time','lat','lon'),mfc_concat)
dk.coords['time']=yrs[0:len(ivt_concat[:,0,0])]
dk.coords['lat']=ivt.lat.values
dk.coords['lon']=ivt.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'synoptic_geochars_jra55gw_MAX.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'synoptic_geochars_jra55gw_MAX.nc',mode='w',format='NETCDF4')

In [ ]:
ds.close()

if 1==1:
    
    ds=xr.open_dataset(dir_data+'synoptic_geochars_era5tp_MAX.nc')
    ivt_concat_era5tp=ds['ivt_concat']
    iwv_concat_era5tp=ds['iwv_concat']
    s_concat_era5tp=ds['s_concat']
    mfc_concat_era5tp=ds['mfc_concat']
    
    ds=xr.open_dataset(dir_data+'synoptic_geochars_era5reid_MAX.nc')
    ivt_concat_era5reid=ds['ivt_concat']
    iwv_concat_era5reid=ds['iwv_concat']
    s_concat_era5reid=ds['s_concat']
    mfc_concat_era5reid=ds['mfc_concat']

    ds=xr.open_dataset(dir_data+'synoptic_geochars_era5mk_MAX.nc')
    ivt_concat_era5mk=ds['ivt_concat']
    iwv_concat_era5mk=ds['iwv_concat']
    s_concat_era5mk=ds['s_concat']
    mfc_concat_era5mk=ds['mfc_concat']

    ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2lo_MAX.nc')
    ivt_concat_merra2lo=ds['ivt_concat']
    iwv_concat_merra2lo=ds['iwv_concat']
    s_concat_merra2lo=ds['s_concat']
    mfc_concat_merra2lo=ds['mfc_concat']

    ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2cndl_MAX.nc')
    ivt_concat_merra2cndl=ds['ivt_concat']
    iwv_concat_merra2cndl=ds['iwv_concat']
    s_concat_merra2cndl=ds['s_concat']
    mfc_concat_merra2cndl=ds['mfc_concat']

    ds=xr.open_dataset(dir_data+'synoptic_geochars_merra2ac_MAX.nc')
    ivt_concat_merra2ac=ds['ivt_concat']
    iwv_concat_merra2ac=ds['iwv_concat']
    s_concat_merra2ac=ds['s_concat']
    mfc_concat_merra2ac=ds['mfc_concat']

    ds=xr.open_dataset(dir_data+'synoptic_geochars_jra55gw_MAX.nc')
    ivt_concat_jra55gw=ds['ivt_concat']
    iwv_concat_jra55gw=ds['iwv_concat']
    s_concat_jra55gw=ds['s_concat']

In [ ]:
lons_west=[x for x in ivt_concat_jra55gw.lon.values if x<-90]
lons_east=[x for x in ivt_concat_jra55gw.lon.values if x>=90]
sel_west=ivt_concat_jra55gw.sel(lon=lons_west)
sel_east=ivt_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,ivt_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_ivt=interp.sel(lon=lons_center)


sel_west=iwv_concat_jra55gw.sel(lon=lons_west)
sel_east=iwv_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,iwv_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_iwv=interp.sel(lon=lons_center)


sel_west=s_concat_jra55gw.sel(lon=lons_west)
sel_east=s_concat_jra55gw.sel(lon=lons_east)
sel_west['lon']=[x+360. for x in lons_west]
sel_east['lon']=[x-360. for x in lons_east]
final=xr.concat([sel_east,s_concat_jra55gw,sel_west],dim='lon')
interp=final.interp(lat=ivt_concat_era5tp.lat.values,lon=ivt_concat_era5tp.lon.values)
lons_center=[x for x in interp.lon.values if -180<=x<180]
fullres_s=interp.sel(lon=lons_center)

In [ ]:
yrs=np.arange(1980,2020,1)
slopes_array_ivt_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_ivt_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_iwv_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_iwv_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_s_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_s_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
slopes_array_mfc_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])
sigs_array_mfc_era5tp=xr.zeros_like(ivt_concat_era5tp[0,:,:])

if 1==1:
    slopes_array_ivt_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_ivt_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_iwv_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_iwv_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_s_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_s_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_mfc_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_mfc_era5reid=xr.zeros_like(ivt_concat_era5tp[0,:,:])

    slopes_array_ivt_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_ivt_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_iwv_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_iwv_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_s_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_s_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_mfc_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_mfc_era5mk=xr.zeros_like(ivt_concat_era5tp[0,:,:])


    slopes_array_ivt_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_ivt_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_iwv_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_iwv_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_s_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_s_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_mfc_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_mfc_merra2lo=xr.zeros_like(ivt_concat_era5tp[0,:,:])

    slopes_array_ivt_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_ivt_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_iwv_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_iwv_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_s_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_s_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_mfc_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_mfc_merra2cndl=xr.zeros_like(ivt_concat_era5tp[0,:,:])

    slopes_array_ivt_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_ivt_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_iwv_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_iwv_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_s_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_s_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_mfc_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_mfc_merra2ac=xr.zeros_like(ivt_concat_era5tp[0,:,:])


    slopes_array_ivt_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_ivt_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_iwv_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_iwv_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    slopes_array_s_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
    sigs_array_s_jra55gw=xr.zeros_like(ivt_concat_era5tp[0,:,:])
size=20
for j in range(ivt_concat_era5tp.lat.size):
    print(j)
    for k in range(ivt_concat_era5tp.lon.size):
        time_series=ivt_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_ivt_era5tp[j,k]=r[0]
            sigs_array_ivt_era5tp[j,k]=r[3]
        else:
            slopes_array_ivt_era5tp[j,k]=np.nan
            sigs_array_ivt_era5tp[j,k]=np.nan
            
        time_series=iwv_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_iwv_era5tp[j,k]=r[0]
            sigs_array_iwv_era5tp[j,k]=r[3]
        else:
            slopes_array_iwv_era5tp[j,k]=np.nan
            sigs_array_iwv_era5tp[j,k]=np.nan
            
        time_series=s_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_s_era5tp[j,k]=r[0]
            sigs_array_s_era5tp[j,k]=r[3]
        else:
            slopes_array_s_era5tp[j,k]=np.nan
            sigs_array_s_era5tp[j,k]=np.nan
            
        time_series=mfc_concat_era5tp[:,j,k].values.tolist()
        defined=[x for x in time_series if x>-1]
        if len(defined)>=size:
            zipped_years=list(zip(yrs,time_series))
            sel_years=[x[0] for x in zipped_years if x[1]>-1]
            sel_values=[x[1] for x in zipped_years if x[1]>-1]
            r=st.linregress(sel_years,sel_values)
            slopes_array_mfc_era5tp[j,k]=r[0]
            sigs_array_mfc_era5tp[j,k]=r[3]
        else:
            slopes_array_mfc_era5tp[j,k]=np.nan
            sigs_array_mfc_era5tp[j,k]=np.nan
            
        ######
        
        if 1==1:
            time_series=ivt_concat_era5reid[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_ivt_era5reid[j,k]=r[0]
                sigs_array_ivt_era5reid[j,k]=r[3]
            else:
                slopes_array_ivt_era5reid[j,k]=np.nan
                sigs_array_ivt_era5reid[j,k]=np.nan

            time_series=iwv_concat_era5reid[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_iwv_era5reid[j,k]=r[0]
                sigs_array_iwv_era5reid[j,k]=r[3]
            else:
                slopes_array_iwv_era5reid[j,k]=np.nan
                sigs_array_iwv_era5reid[j,k]=np.nan

            time_series=s_concat_era5reid[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_s_era5reid[j,k]=r[0]
                sigs_array_s_era5reid[j,k]=r[3]
            else:
                slopes_array_s_era5reid[j,k]=np.nan
                sigs_array_s_era5reid[j,k]=np.nan

            time_series=mfc_concat_era5reid[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_mfc_era5reid[j,k]=r[0]
                sigs_array_mfc_era5reid[j,k]=r[3]
            else:
                slopes_array_mfc_era5reid[j,k]=np.nan
                sigs_array_mfc_era5reid[j,k]=np.nan

            #####

            time_series=ivt_concat_era5mk[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_ivt_era5mk[j,k]=r[0]
                sigs_array_ivt_era5mk[j,k]=r[3]
            else:
                slopes_array_ivt_era5mk[j,k]=np.nan
                sigs_array_ivt_era5mk[j,k]=np.nan

            time_series=iwv_concat_era5mk[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_iwv_era5mk[j,k]=r[0]
                sigs_array_iwv_era5mk[j,k]=r[3]
            else:
                slopes_array_iwv_era5mk[j,k]=np.nan
                sigs_array_iwv_era5mk[j,k]=np.nan

            time_series=s_concat_era5mk[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_s_era5mk[j,k]=r[0]
                sigs_array_s_era5mk[j,k]=r[3]
            else:
                slopes_array_s_era5mk[j,k]=np.nan
                sigs_array_s_era5mk[j,k]=np.nan

            time_series=mfc_concat_era5mk[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_mfc_era5mk[j,k]=r[0]
                sigs_array_mfc_era5mk[j,k]=r[3]
            else:
                slopes_array_mfc_era5mk[j,k]=np.nan
                sigs_array_mfc_era5mk[j,k]=np.nan

            ##########

            time_series=ivt_concat_merra2lo[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_ivt_merra2lo[j,k]=r[0]
                sigs_array_ivt_merra2lo[j,k]=r[3]
            else:
                slopes_array_ivt_merra2lo[j,k]=np.nan
                sigs_array_ivt_merra2lo[j,k]=np.nan

            time_series=iwv_concat_merra2lo[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_iwv_merra2lo[j,k]=r[0]
                sigs_array_iwv_merra2lo[j,k]=r[3]
            else:
                slopes_array_iwv_merra2lo[j,k]=np.nan
                sigs_array_iwv_merra2lo[j,k]=np.nan

            time_series=s_concat_merra2lo[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_s_merra2lo[j,k]=r[0]
                sigs_array_s_merra2lo[j,k]=r[3]
            else:
                slopes_array_s_merra2lo[j,k]=np.nan
                sigs_array_s_merra2lo[j,k]=np.nan

            time_series=mfc_concat_merra2lo[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_mfc_merra2lo[j,k]=r[0]
                sigs_array_mfc_merra2lo[j,k]=r[3]
            else:
                slopes_array_mfc_merra2lo[j,k]=np.nan
                sigs_array_mfc_merra2lo[j,k]=np.nan

            #####

            time_series=ivt_concat_merra2cndl[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_ivt_merra2cndl[j,k]=r[0]
                sigs_array_ivt_merra2cndl[j,k]=r[3]
            else:
                slopes_array_ivt_merra2cndl[j,k]=np.nan
                sigs_array_ivt_merra2cndl[j,k]=np.nan

            time_series=iwv_concat_merra2cndl[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_iwv_merra2cndl[j,k]=r[0]
                sigs_array_iwv_merra2cndl[j,k]=r[3]
            else:
                slopes_array_iwv_merra2cndl[j,k]=np.nan
                sigs_array_iwv_merra2cndl[j,k]=np.nan

            time_series=s_concat_merra2cndl[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_s_merra2cndl[j,k]=r[0]
                sigs_array_s_merra2cndl[j,k]=r[3]
            else:
                slopes_array_s_merra2cndl[j,k]=np.nan
                sigs_array_s_merra2cndl[j,k]=np.nan

            time_series=mfc_concat_merra2cndl[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_mfc_merra2cndl[j,k]=r[0]
                sigs_array_mfc_merra2cndl[j,k]=r[3]
            else:
                slopes_array_mfc_merra2cndl[j,k]=np.nan
                sigs_array_mfc_merra2cndl[j,k]=np.nan

            #####

            time_series=ivt_concat_merra2ac[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_ivt_merra2ac[j,k]=r[0]
                sigs_array_ivt_merra2ac[j,k]=r[3]
            else:
                slopes_array_ivt_merra2ac[j,k]=np.nan
                sigs_array_ivt_merra2ac[j,k]=np.nan

            time_series=iwv_concat_merra2ac[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_iwv_merra2ac[j,k]=r[0]
                sigs_array_iwv_merra2ac[j,k]=r[3]
            else:
                slopes_array_iwv_merra2ac[j,k]=np.nan
                sigs_array_iwv_merra2ac[j,k]=np.nan

            time_series=s_concat_merra2ac[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_s_merra2ac[j,k]=r[0]
                sigs_array_s_merra2ac[j,k]=r[3]
            else:
                slopes_array_s_merra2ac[j,k]=np.nan
                sigs_array_s_merra2ac[j,k]=np.nan

            time_series=mfc_concat_merra2ac[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_mfc_merra2ac[j,k]=r[0]
                sigs_array_mfc_merra2ac[j,k]=r[3]
            else:
                slopes_array_mfc_merra2ac[j,k]=np.nan
                sigs_array_mfc_merra2ac[j,k]=np.nan

            ############

            time_series=fullres_ivt[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_ivt_jra55gw[j,k]=r[0]
                sigs_array_ivt_jra55gw[j,k]=r[3]
            else:
                slopes_array_ivt_jra55gw[j,k]=np.nan
                sigs_array_ivt_jra55gw[j,k]=np.nan

            time_series=fullres_iwv[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_iwv_jra55gw[j,k]=r[0]
                sigs_array_iwv_jra55gw[j,k]=r[3]
            else:
                slopes_array_iwv_jra55gw[j,k]=np.nan
                sigs_array_iwv_jra55gw[j,k]=np.nan

            time_series=fullres_s[:,j,k].values.tolist()
            defined=[x for x in time_series if x>-1]
            if len(defined)>=size:
                zipped_years=list(zip(yrs,time_series))
                sel_years=[x[0] for x in zipped_years if x[1]>-1]
                sel_values=[x[1] for x in zipped_years if x[1]>-1]
                r=st.linregress(sel_years,sel_values)
                slopes_array_s_jra55gw[j,k]=r[0]
                sigs_array_s_jra55gw[j,k]=r[3]
            else:
                slopes_array_s_jra55gw[j,k]=np.nan
                sigs_array_s_jra55gw[j,k]=np.nan    

In [ ]:
test=sigs_array_ivt_era5tp

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(test.lon,test.lat,test,vmin=0,vmax=1,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
import metpy.calc as mpcalc

test=slopes_array_mfc_era5tp

#test=mpcalc.smooth_n_point(test,n=9,passes=50)

annual=mfc_concat_era5tp
mean=annual.mean(dim='time',skipna=True)
percent=test*40./mean*100.

weights=np.cos(np.deg2rad(percent.lat))
weighted=percent.weighted(weights)
mean1=weighted.mean().values.tolist()
print(mean1)

fig=plt.figure(figsize=(20,10))
ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
ax.pcolormesh(percent.lon,percent.lat,percent,vmin=-30,vmax=30,cmap=plt.cm.seismic)
ax.coastlines(resolution='10m')
plt.show()

In [ ]:
dk=xr.Dataset()
dk['slopes_array_ivt_era5tp']=(('lat','lon'),slopes_array_ivt_era5tp.values)
dk['slopes_array_ivt_era5reid']=(('lat','lon'),slopes_array_ivt_era5reid.values)
dk['slopes_array_ivt_era5mk']=(('lat','lon'),slopes_array_ivt_era5mk.values)
dk['slopes_array_ivt_merra2lo']=(('lat','lon'),slopes_array_ivt_merra2lo.values)
dk['slopes_array_ivt_merra2cndl']=(('lat','lon'),slopes_array_ivt_merra2cndl.values)
dk['slopes_array_ivt_merra2ac']=(('lat','lon'),slopes_array_ivt_merra2ac.values)
dk['slopes_array_ivt_jra55gw']=(('lat','lon'),slopes_array_ivt_jra55gw.values)

dk['sigs_array_ivt_era5tp']=(('lat','lon'),sigs_array_ivt_era5tp.values)
dk['sigs_array_ivt_era5reid']=(('lat','lon'),sigs_array_ivt_era5reid.values)
dk['sigs_array_ivt_era5mk']=(('lat','lon'),sigs_array_ivt_era5mk.values)
dk['sigs_array_ivt_merra2lo']=(('lat','lon'),sigs_array_ivt_merra2lo.values)
dk['sigs_array_ivt_merra2cndl']=(('lat','lon'),sigs_array_ivt_merra2cndl.values)
dk['sigs_array_ivt_merra2ac']=(('lat','lon'),sigs_array_ivt_merra2ac.values)
dk['sigs_array_ivt_jra55gw']=(('lat','lon'),sigs_array_ivt_jra55gw.values)


dk['slopes_array_iwv_era5tp']=(('lat','lon'),slopes_array_iwv_era5tp.values)
dk['slopes_array_iwv_era5reid']=(('lat','lon'),slopes_array_iwv_era5reid.values)
dk['slopes_array_iwv_era5mk']=(('lat','lon'),slopes_array_iwv_era5mk.values)
dk['slopes_array_iwv_merra2lo']=(('lat','lon'),slopes_array_iwv_merra2lo.values)
dk['slopes_array_iwv_merra2cndl']=(('lat','lon'),slopes_array_iwv_merra2cndl.values)
dk['slopes_array_iwv_merra2ac']=(('lat','lon'),slopes_array_iwv_merra2ac.values)
dk['slopes_array_iwv_jra55gw']=(('lat','lon'),slopes_array_iwv_jra55gw.values)

dk['sigs_array_iwv_era5tp']=(('lat','lon'),sigs_array_iwv_era5tp.values)
dk['sigs_array_iwv_era5reid']=(('lat','lon'),sigs_array_iwv_era5reid.values)
dk['sigs_array_iwv_era5mk']=(('lat','lon'),sigs_array_iwv_era5mk.values)
dk['sigs_array_iwv_merra2lo']=(('lat','lon'),sigs_array_iwv_merra2lo.values)
dk['sigs_array_iwv_merra2cndl']=(('lat','lon'),sigs_array_iwv_merra2cndl.values)
dk['sigs_array_iwv_merra2ac']=(('lat','lon'),sigs_array_iwv_merra2ac.values)
dk['sigs_array_iwv_jra55gw']=(('lat','lon'),sigs_array_iwv_jra55gw.values)


dk['slopes_array_s_era5tp']=(('lat','lon'),slopes_array_s_era5tp.values)
dk['slopes_array_s_era5reid']=(('lat','lon'),slopes_array_s_era5reid.values)
dk['slopes_array_s_era5mk']=(('lat','lon'),slopes_array_s_era5mk.values)
dk['slopes_array_s_merra2lo']=(('lat','lon'),slopes_array_s_merra2lo.values)
dk['slopes_array_s_merra2cndl']=(('lat','lon'),slopes_array_s_merra2cndl.values)
dk['slopes_array_s_merra2ac']=(('lat','lon'),slopes_array_s_merra2ac.values)
dk['slopes_array_s_jra55gw']=(('lat','lon'),slopes_array_s_jra55gw.values)

dk['sigs_array_s_era5tp']=(('lat','lon'),sigs_array_s_era5tp.values)
dk['sigs_array_s_era5reid']=(('lat','lon'),sigs_array_s_era5reid.values)
dk['sigs_array_s_era5mk']=(('lat','lon'),sigs_array_s_era5mk.values)
dk['sigs_array_s_merra2lo']=(('lat','lon'),sigs_array_s_merra2lo.values)
dk['sigs_array_s_merra2cndl']=(('lat','lon'),sigs_array_s_merra2cndl.values)
dk['sigs_array_s_merra2ac']=(('lat','lon'),sigs_array_s_merra2ac.values)
dk['sigs_array_s_jra55gw']=(('lat','lon'),sigs_array_s_jra55gw.values)


dk['slopes_array_mfc_era5tp']=(('lat','lon'),slopes_array_mfc_era5tp.values)
dk['slopes_array_mfc_era5reid']=(('lat','lon'),slopes_array_mfc_era5reid.values)
dk['slopes_array_mfc_era5mk']=(('lat','lon'),slopes_array_mfc_era5mk.values)
dk['slopes_array_mfc_merra2lo']=(('lat','lon'),slopes_array_mfc_merra2lo.values)
dk['slopes_array_mfc_merra2cndl']=(('lat','lon'),slopes_array_mfc_merra2cndl.values)
dk['slopes_array_mfc_merra2ac']=(('lat','lon'),slopes_array_mfc_merra2ac.values)

dk['sigs_array_mfc_era5tp']=(('lat','lon'),sigs_array_mfc_era5tp.values)
dk['sigs_array_mfc_era5reid']=(('lat','lon'),sigs_array_mfc_era5reid.values)
dk['sigs_array_mfc_era5mk']=(('lat','lon'),sigs_array_mfc_era5mk.values)
dk['sigs_array_mfc_merra2lo']=(('lat','lon'),sigs_array_mfc_merra2lo.values)
dk['sigs_array_mfc_merra2cndl']=(('lat','lon'),sigs_array_mfc_merra2cndl.values)
dk['sigs_array_mfc_merra2ac']=(('lat','lon'),sigs_array_mfc_merra2ac.values)

dk.coords['lat']=slopes_array_ivt_era5tp.lat.values
dk.coords['lon']=slopes_array_ivt_era5tp.lon.values

dk['lat'].attrs["units"]='degrees_north'
dk['lon'].attrs["units"]='degrees_east'

try:
    os.remove(dir_data+'synoptic_geotrends_arrays_artmip_MAX.nc')
except OSError:
    pass
dk.to_netcdf(dir_data+'synoptic_geotrends_arrays_artmip_MAX.nc',mode='w',format='NETCDF4')